In [1]:
import numpy as np
import pandas as pd


# Structural Econometrics in Labor and IO.

# Problem Set: Estimation of Static Differentiated Product Demand Models.

  
## Description

Consider a scenario where only market level data on product demand are available: market shares, prices, product characteristics. We wish to estimate consumer preferences, compute the resulting price elasticities, and ultimately learn the firms' implied profit margins and marginal cost. To do so, we make use of a model of individual utility maximization and imperfect competition on the product market. Note, that the crucial assumption in Berry (1994) and virtually all subsequent work is that market shares are generated by a large number (technically, infinity) of consumers in the market and, hence, can be used as a measure of individual choice probabilities.



#### Demand
We assume consumer $i$ chooses one unit of product $j\in J$ or an outside good $j=0$ (e.g. no purchase) to obtain utility

\begin{equation}
u_{ijt} = x_{jt}\beta_i + \alpha p_{jt} + \xi_{jt} + \varepsilon_{ijt} = \delta_{jt} + \mu_{jt}(\nu_i) + \varepsilon_{ijt}\tag{1}
\end{equation}

where $(x_{jt},p_{jt})$ are observable characteristics and price, $\xi_{jt}$ is the unobservable characteristic, $\delta_{jt} = x_{jt}\beta + \alpha p_{jt} + \xi_{jt}$, $\mu_{jt}(\nu_i) = x_{jt}\sigma \nu_i$, and $\varepsilon_{ijt}$ i.i.d. extreme value type 1.  The utility of the outside good is normalized such that $\delta_{0t}=0$. The assumption of utility-maximizing consumers and the distribution of $\varepsilon_{ijt}$ yields the logit choice probabilities:

\begin{equation}
s_{jt}(\delta_t,\sigma) = \int \frac{\exp(\delta_{jt} + \mu_{jt}(\nu_i))}{1+\sum_{l=1}^J \exp(\delta_{lt} + \mu_{lt}(\nu_i))} dP_{\nu}(\nu),
\tag{2}
\end{equation}

where $P_{\nu}$ is typically assumed to take a specific parametric form.

#### Supply
Assume that firms f maximize profits given by

\begin{equation}
\pi_f = \sum_{k\in F_f}(p_{kt} - c_{kt})s_{kt}L_{t},
\tag{3}
\end{equation}

where $c_{kt}$ are marginal cost and $L_{t}$ market size, so that the system of FOC for a Nash equilibrium:

\begin{equation}
s_{jt} + \sum_{k\in F_f}(p_{kt} - c_{kt})\frac{\partial s_{kt}}{\partial p_{jt}} = 0
\tag{4}
\end{equation}

For implementation, it will be useful to write expression (4) in vector notation as $s_t+\Delta_t(p_t-c_t)=0,$ where $\Delta_t(j,k)$ denotes a diagonal matrix of own-price derivatives and off-diagonal elements according to market structure (equal to zero in the case of single-product firms). If marginal cost are known, we obtain the supply side by solving the system for $c_t$:

\begin{equation}
p_t+\Delta_t^{-1}s_t = c_t
\tag{5}
\end{equation}

When changes in market structure are of interest (e.g. merger analysis), it is useful to generate a full $\tilde{\Delta}_t$ derivative matrix, an equally-sized matrix $O_t$ designating firms' product ownership, and element-wise multiply $\tilde{\Delta}_t$ and $O_t$. $\partial s_{kt} / \partial p_{jt}$ is given by $\alpha s_{jt} (1-s_{jt})$ if $j=k$ and $-\alpha s_{jt} s_{kt}$ otherwise.

## Problems


### 1. Simulation

Simulate data based on the described model assuming the following

- $J=10$ products are sold in $T=250$ markets (size $L_t=1$) by single-product firms.
- Two observable product characteristics $x_{jt}=(1,x_{jt}^1)$, with $x_{jt}^1 \sim U(1,2)$.
- Marginal cost $c_{jt}=x_{jt}\gamma_1 + w_{jt}\gamma_2 + \omega_{jt}$.
- Three observable cost shifters $w_{jt}=(w_{jt}^1,w_{jt}^2,w_{jt}^3)$, all i.i.d. $U(0,1)$.
- Marginal cost parameters: $\gamma_1=(0.7,0.7)$ and $\gamma_2=(1,1,1)$.
- Unobserved demand and cost characteristic $(\xi_{jt},\omega_{jt}) \sim N(0,\sigma_c)$ 
    with $\sigma_c = \left[ \begin{array}{cc} 1 & 0.7 \\ 0.7 & 1 \\ \end{array}\right]$.  
- Preference parameters $\beta=(2,2), \alpha=-2, \sigma=(0,1)$.
- $\nu \sim N(0,1)$.
-  $L_t=1$ in each market.

To simulate the data, you must code functions computing


- a $T\times J$ matrix containing market shares using equation (4). This requires numerical integration, which can be done for example in the following ways:


    - Monte Carlo integration: Draw many (at least a 100, but of course depends on the function) values from the distribution over which you want to integrate, calculate the value of the function and take the mean over all integration points. 


    - Quadrature: You can think of this of a smart way to draw the values from the distribution, which drastically reduces the number of draws. The draws are then not equally weighted (as in the mean) and instead they have particular weights. You can find the mathematical details here: https://en.wikipedia.org/wiki/Gauss–Hermite_quadrature and the appropriate scipy function is *scipy.special.roots_hermite*. 
    
  
- a $T\times (J \times J)$ matrix containing market share derivatives with respect to price (own-price on the diagonal and cross-price off the diagonal),
- a $T\times J$ matrix containing (Bertrand-Nash) equilibrium prices, using equation (5) and scipy's root-finding function scipy's root-finding function *scipy.optimize.fsolve* or *scipy.optimize.root*.

### 2. Estimation

The simulated data at hand, forget the parameters specified in Problem 1 and

- assume that $\sigma = (0,0)$. Estimate $\{\alpha,\beta\}$ using OLS and report the results.

- still assume that $\sigma = (0,0)$. Estimate $\{\alpha,\beta\}$ using linear IV and report the results for two alternative sets of instrumental variables: 

    a) $x_{jt}$, the squared term $(x_{jt}^1)^2$, and BLP instruments (sums of other firms' $x_{jt}$)

- still assume that $\sigma = (0,0)$. Estimate $\{\alpha,\beta\}$ using linear IV and report the results for two alternative sets of instrumental variables:     

    b) all variables in a) plus cost shifters ($w_{jt}$).

### 3. Contraction mapping

Code the BLP contraction mapping to obtain $\delta$ for a given set of parameters (requires observed market shares and the market share function as input): $\delta_t=s_t^{-1}(s_t,\sigma) \equiv \delta_t(s_t,\sigma)$. The algorithm should iterate as follows: $\delta^{h+1} = \delta^h + \ln (s) - \ln (s(\delta^h,\sigma))$.

### 4. Estimation with contraction mapping

Build the function that nests the $\delta$ contraction mapping and the GMM objective function. Then, find the parameter values that minimize this function *scipy.optimize.minimize*.